## <center> American Express Case </center>

### Import Data and Storage in Hash Table

In [40]:
import numpy as np
import pandas as pd
import os
import re

In [41]:
#Variables
FLAG_TABLE = False

In [42]:
path = os.getcwd()
folder = "output_amex_documentai"
folder_file = "file_2"
files = os.listdir(os.path.join(folder, folder_file))

In [43]:
dic_data = {}
for file in files:
    data = pd.read_csv(os.path.join(path, folder, folder_file, file))
    p = "(pg\d\w+)"
    regex_p= re.compile(p)
    dic_key = regex_p.search(file).group()
    dic_data[dic_key] = data

In [44]:
for key in dic_data.keys():
    print(key)

pg1_tb0
pg1_tb1
pg1_tb2
pg3_tb0
pg3_tb1
pg3_tb2
pg3_tb3
pg3_tb4
pg3_tb5
pg4_tb0
pg4_tb1
pg4_tb2


In [51]:
dic_data["pg4_tb1"]

,Detail,Continued *Indicates posting date,Unnamed: 2,- denotes Pay Over Time and/or Cash Advance activity
0,NaN,NaN,NaN,Amount
1,11/21/21,0200 FOREVER 21 0000 954-837-0335,SUNRISE,FL $45.98
2,11/26/21,SAM'S CLUB 6341 6341 WHOLESALE CLUB,SUNRISE,FL $168.19
3,11/29/21,276 INDIAN TRCE 57543608806 9542178456,WESTON,FL $43.16
4,SOFIA Card,GRIMBLAT Ending 0-41021,NaN,Amount
5,11/18/21,PUBLIX #583 000000583 9543844190,WESTON,FL $18.55
6,11/21/21,SHIBUYA SUSHI ART WESTON 0000 954-756-8255,FORT LAUDERDA,FL $69.66


### Separate used table and Storage in Another Dic

In [7]:
tables = {}
for key in dic_data.keys():
    if len(dic_data[key].columns) == 4 and "Balance" not in dic_data[key].columns:
        tables[key] = dic_data[key] 
    # elif len(dic_data[key].columns) == 3 and "Amount" in dic_data[key].columns:
    #     tables[key] = dic_data[key]


In [8]:
for key in tables.keys():
    print(key)

2022-08-07_pg3_tb0
2022-08-07_pg4_tb0


In [9]:
dic_data["2022-08-07_pg3_tb0"]

,Card,Ending 1-62000,Unnamed: 2,Foreign Spend Amount
0,07/09/22,PARKING PAY PHONE 0814 305-373-6789,MIAMI,FL $2.27
1,07/10/22,TARGET 010389 09100010389 612-3044357,NORTH MIAMI B,FL $37.29
2,07/10/22,STARBUCKS STORE 6191 FAST FOOD RESTAURANT,NORTH MIAMI B,FL $15.46
3,07/10/22,PUBLIX #1017 000001017 8636881188,NORTH MIAMI,FL $25.34
4,07/10/22,PARKING PAY PHONE 0814 305-373-6789,MIAMI FL,$2.27
5,07/11/22,FAROFA OF NORTH MIAMI 0000 305-947-3098,NORTH MIAMI,FL $29.84
6,07/12/22,PUBLIX 3059313761,MIAMI,FL $14.66
7,07/12/22,PREMIER ONE LOW PRICE 305-948-0740,NORTH MIAMI,FL $37.32
8,07/17/22,UBER EATS 8005928996,SAN FRANSISCO,CA $50.91
9,07/17/22,UBER TRIP HELP.UBER.COM,SAN FRANCISCO,775.99 Mexican Pesos $37.70


### Preprocessing tables 

In [10]:
tables_2 = {}
for key in tables.keys():
    if len(tables[key].columns) == 4:
        
        table = tables[key].astype(str)
        table.columns = ["DATE", "DESCRIPTION", "column_not_name", "AMOUNT"]
        
        patron = "(\$)(\d*|\d{1,3}(,\d{3})*)(\.\d+)"
        regex_number_extractor = re.compile(patron)
        table["AMOUNT"] = table["AMOUNT"].apply(lambda x: regex_number_extractor.search(x))
        table["AMOUNT"] = table["AMOUNT"].apply(lambda x: x if x is None else x.group())
        
        table = table.drop("column_not_name", axis=1)
        
        table.AMOUNT = table.AMOUNT.apply(lambda x: x.replace("$", ""))
        table["AMOUNT"] = table["AMOUNT"].astype(float)
        table["type"] = "Withdrawals"
        
        tables_2[key] = table

    # else:
        
    #     table = tables[key].astype(str)
    #     table.columns = ["DATE", "DESCRIPTION", "AMOUNT"]
        
    #     patron = "(\$)(\d*|\d{1,3}(,\d{3})*)(\.\d+)"
    #     regex_number_extractor = re.compile(patron)
    #     table["AMOUNT"] = table["AMOUNT"].apply(lambda x: regex_number_extractor.search(x))
    #     table["AMOUNT"] = table["AMOUNT"].apply(lambda x: x if x is None else x.group())
        
    #     table = table.drop(table.index[len(table)-1])
    #     table = table.dropna()
        
    #     table.AMOUNT = table.AMOUNT.apply(lambda x: x.replace("$", ""))
    #     table["AMOUNT"] = table["AMOUNT"].astype(float)
    #     table["type"] = "Withdrawals"

    #     tables_2[key] = table

In [11]:
for key in tables_2.keys():
    print(key)

2022-08-07_pg3_tb0
2022-08-07_pg4_tb0


In [12]:
tables_2["2022-08-07_pg4_tb0"]

,DATE,DESCRIPTION,AMOUNT,type
0,07/17/22,UBER EATS 8005928996,9.16,Withdrawals
1,07/17/22,SUNPASS OPERATIONS 888-865-5352,112.50,Withdrawals
2,07/19/22,DR JOSE J ALVAREZ AND ASS 000000001 3059485002,92.00,Withdrawals
3,07/22/22,SUNNY ISLES PARKING METER 305-792-1707,4.27,Withdrawals
4,07/24/22,UBER TRIP HELP.UBER.COM,1.12,Withdrawals
5,07/24/22,UBER EATS 8005928996,138.81,Withdrawals
6,07/24/22,UBER EATS 8005928996,24.98,Withdrawals
7,07/25/22,NETFLIX.COM NETFLIX.COM,21.61,Withdrawals
8,07/26/22,PARKING PAY PHONE 0814 305-373-6789,2.27,Withdrawals
9,07/26/22,NOVECENTO RESTAURANT,27.72,Withdrawals


### Final Step and Compile all Information in 1 DataFrame

In [13]:
df_american_express_final = pd.DataFrame()
for key in tables_2.keys():
    df_american_express_final = pd.concat([df_american_express_final, tables_2[key]], axis=0)
    df_american_express_final = df_american_express_final.sort_values(["type", "DATE"])
    
    

### Check Information

In [14]:
df_american_express_final

,DATE,DESCRIPTION,AMOUNT,type
0,07/09/22,PARKING PAY PHONE 0814 305-373-6789,2.27,Withdrawals
1,07/10/22,TARGET 010389 09100010389 612-3044357,37.29,Withdrawals
2,07/10/22,STARBUCKS STORE 6191 FAST FOOD RESTAURANT,15.46,Withdrawals
3,07/10/22,PUBLIX #1017 000001017 8636881188,25.34,Withdrawals
4,07/10/22,PARKING PAY PHONE 0814 305-373-6789,2.27,Withdrawals
5,07/11/22,FAROFA OF NORTH MIAMI 0000 305-947-3098,29.84,Withdrawals
6,07/12/22,PUBLIX 3059313761,14.66,Withdrawals
7,07/12/22,PREMIER ONE LOW PRICE 305-948-0740,37.32,Withdrawals
8,07/17/22,UBER EATS 8005928996,50.91,Withdrawals
9,07/17/22,UBER TRIP HELP.UBER.COM,37.70,Withdrawals


In [15]:
df_american_express_final[df_american_express_final.type == "Income"].AMOUNT.sum()

0.0

In [16]:
df_american_express_final[df_american_express_final.type == "Withdrawals"].AMOUNT.sum()

692.77

## Delete Names

In [17]:
names = ["JOSE J ALVAREZ"]

In [18]:
for name in names:
    df_american_express_final.DESCRIPTION = df_american_express_final.DESCRIPTION.apply(lambda x: x.replace(name, "#####"))

## Save Output

In [19]:
df_american_express_final["bank"] = "American Express"
folder_output = "output_script"
file = "american_express_output.csv"

In [20]:
df_american_express_final.to_csv(os.path.join(path,folder_output, file), index=False, encoding="utf8")